In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
from pathlib import Path
import yaml
import torch

# sys.path.append(os.path.abspath(".."))

# from rade_gs.models import RadegsModel, RadegsModelConfig
from nerfstudio.models.splatfacto import SplatfactoModel, SplatfactoModelConfig
from nerfstudio.utils.eval_utils import eval_setup
from nerfstudio.data.scene_box import OrientedBox, SceneBox

[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/02/25 18:15:34.958 6296] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
# Find config for our current model
load_config = Path('/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/feature-splatting/2025-06-18_174658/config.yml')

config = yaml.load(load_config.read_text(), Loader=yaml.Loader)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = config.pipeline.setup(device='cuda', test_mode='inference')

aabb_scale = pipeline.datamanager.dataparser.config.scene_scale

scene_box = SceneBox(
    aabb=torch.tensor(
        [[-aabb_scale, -aabb_scale, -aabb_scale], [aabb_scale, aabb_scale, aabb_scale]], dtype=torch.float32
    )
)

train_samples = len(pipeline.datamanager.train_dataset)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[13:35:09] use color only optimization with sigmoid activation                                         ]8;id=132383;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=732632;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

100%|███████████████████████████████████████| 891M/891M [02:51<00:00, 5.45MiB/s]


In [5]:
config = RadegsModelConfig(output_depth_during_training=True)
radegs_model = RadegsModel(config, scene_box=scene_box, num_train_data=train_samples)

In [5]:
idx = 0
camera = pipeline.datamanager.train_dataset.cameras[idx:idx+1]

In [11]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version PyTorch was compiled with: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

PyTorch version: 2.1.2+cu118
CUDA available: True
CUDA version PyTorch was compiled with: 11.8
Number of GPUs: 1


In [7]:
# Before calling radegs_model.get_outputs(camera)
print(f"Camera device: {camera.device if hasattr(camera, 'device') else 'No device attr'}")

# Check model parameters
for name, param in radegs_model.named_parameters():
    if param.device.type != 'cuda':
        print(f"Parameter {name} is on {param.device}")
        break

Camera device: cpu
Parameter device_indicator_param is on cpu


In [12]:
# Move the entire model to CUDA
radegs_model = radegs_model.cuda()

# Or if you want to be explicit about the device
device = torch.device('cuda:0')
radegs_model = radegs_model.to(device)

camera = camera.to('cuda:0')

In [16]:
radegs_model._downscale_if_required()

torch.Size([120, 67, 3])

In [53]:
outputs = radegs_model.get_outputs(camera)

In [62]:
fs_outputs = pipeline.model.get_outputs(camera)

In [60]:
outputs['info'].keys()

dict_keys(['camera_ids', 'gaussian_ids', 'radii', 'means2d', 'depths', 'conics', 'opacities', 'tile_width', 'tile_height', 'tiles_per_gauss', 'isect_ids', 'flatten_ids', 'isect_offsets', 'width', 'height', 'tile_size', 'n_cameras'])